<a id="top"></a>
# Single Lens Fitting & Pipelines

***

<a href="https://colab.research.google.com/github/rges-pit/data-challenge-notebooks/blob/main/AAS%20Workshop/Session%20B:%20Single%20Lens%20%26%20Pipelines/Single_Lens_Pipeline.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Learning Goals

By the end of this session, you will be able to:

- Configure a reproducible environment (local, Colab, or Nexus) for the single-lens workflow.
- Inspect Roman Data Challenge light curves and understand the inputs required for `MulensModel`.
- Fit single-lens microlensing events **at scale**.
- Structure your code for **repeatability and automation**.
- Apply **priors** that penalize unphysical solutions.
- Detect when a pipeline is breaking—and treat that as actionable feedback instead of failure.

In addition, you'll gain:
- Hands-on experience with OGLE Early Warning System (EWS) data.
- Practice writing your own event-finding heuristics.
- An optional challenge that fits a full Roman observing season, tying the workflow together.

## Introduction
This session adapts the RGES-PIT minicourse material for the Roman Microlensing Data Challenge 2026, the AAS workshop, and self-directed learning. You will load challenge-provided light curves and run point-source point-lens fits with `MulensModel`. 

### Why Bulk Single-Lens Pipelines?

The goal is to develop infrastructure that lets you process dozens—eventually thousands—of events automatically while still catching subtle astrophysics.

### Why Are We Doing This?

Simulations suggest the Roman Space Telescope will deliver tens of thousands of microlensing events. Manual, hand-tuned fits will not scale. Building lightweight pipelines that can stream data, test models, and flag interesting events is how we keep up with the volume.

### What's in It for You?

You will simulate processing a full observing season: downloading OGLE/Roman data, defining objective functions, writing event finders, applying priors, parallelizing computation, and adapting to subtle data challenges in real time. This is as much about workflow discipline as it is about models.

### Microlensing Parameter Refresher
A point-source point-lens (PSPL) event is described by a compact set of geometric and photometric parameters:

- **$t_0$** – time of closest approach between the source and lens lines of sight.
- **$u_0$** – impact parameter in units of the angular Einstein radius ($\theta_E$). Values $|u_0| \ll 1$ yield high magnifications.
- **$t_E$** – Einstein timescale, i.e., the time required for the lens-source separation to change by $\theta_E$; it sets the event duration.
- **$F_s$** – unmagnified source flux in the observed bandpass.
- **$F_b$** – blended flux from unrelated stars or the lens itself.

Optional higher-order terms show up often in Roman analyses:
- **$\rho$** – source radius normalized by $\theta_E$, needed when finite-source effects smear the peak.
- **$\pi_E$** – microlens parallax vector (north/east components) that captures the projected lens-source parallax.

The instantaneous magnification for a PSPL event is
$$
A(u) = \frac{u^2 + 2}{u\sqrt{u^2 + 4}}, \qquad u^2 = u_0^2 + \left(\frac{t - t_0}{t_E}\right)^2.
$$
Packages like `MulensModel` will evaluate $A(u)$ and its finite-source/parallax extensions for us; the rest of this notebook shows how to fit $t_0$, $u_0$, $t_E$, $F_s$, $F_b$, and (optionally) $\rho$ or $\pi_E$ for bulk datasets.

For a more robust introduction to the theory of microlensing, refer to the learning resources listed on the [RGES-PIT website](https://rges-pit.org/resources/#:~:text=Background%20material,Jupyter%20Notebook%20Course%20%C2%A0).

### Notebook Contents

The workflow for this notebook consists of:
* [Environment Setup](#Environment-Setup)
  - [Imports](#Imports)
* [Single-Lens Fitting with MulensModel](#single-lens-fitting-with-mulensmodel)
* [OGLE EWS Bulk Light Curve Fit](#ogle-ews-bulk-light-curve-fit)
  - [Getting the Ground-Based Data](#getting-the-ground-based-data)
  - [Fitting a Basic PSPL Model](#fitting-a-basic-pspl-model)
  - [Priors](#priors)
  - [Parallel Processing](#parallel-processing)
  - [Custom Event Finder](#custom-event-finder)
  - [Advanced Modeling Techniques and Higher-Order Models](#advanced-modeling-techniques-and-higher-order-models)
* [Full Season Roman Fit](#full-season-roman-fit)
  - [Nexus Data Access](#nexus-data-access)
  - [Alternate Data Access](#alternative-data-access)
  - [Load and Sort the Data](#load-and-sort-the-data)
  - [Adjusting the Model for an L2 Orbit](#adjusting-the-model-for-an-l2-orbit)
  - [Run the Fits](#run-the-fits)
  - [Evaluate Your Results](#evaluate-your-results)
* [Exercises and Next Steps](#Exercises)
* [Additional Resources](#additional-resources)
* [About this Notebook](#about-this-notebook)

## Environment Setup

Choose whichever platform best matches your workflow:

- **Colab** provides a zero-install option for quick experimentation.
- **Roman Research Nexus** users should activate the `rges-pit-dc` kernel; all required packages are pre-installed there.
- **Local** environments can be provisioned with the accompanying `env.yml` file for parity with the Nexus image.

If you are new to Jupyter notebooks, the [Getting Started guide](https://medium.com/codingthesmartway-com-blog/getting-started-with-jupyter-notebook-for-python-4e7082bd5d46) offers a quick primer before you dive into this workflow.


<!-- NEXUS-ONLY -->
> ### Running on the Roman Research Nexus
> If you are following this notebook on the Roman Research Nexus (RRN):
> 1. Open the **Kernel > Change Kernel** menu and select `rges-pit-dc` (will be published with the workshop).
> 1. Run `!source kernel-activate rges-pit-dc` once per session to ensure the correct environment is active.
> 1. Store any files you generate in your home directory or team space; the `/roman/notebooks` directory is read-only.

In [ ]:
# NEXUS-ONLY
%source kernel-activate rges-pit-dc

Please **save this notebook in a space you control** (GitHub fork, local repo, Nexus team storage, or Google Drive) so you can return to it later. Hosted environments such as Colab or the Nexus will not auto-save changes to the canonical repository.

### Imports

For this notebook, we rely on a standard scientific Python stack plus `MulensModel`, `pathos`, and a few utilities for working with OGLE/Roman data.

In [ ]:
#@title Imports and Setup

# system tools
import os
import sys
from io import StringIO
import time
from typing import Tuple, Callable, Optional, List
import shutil
from pathlib import Path

# data analysis tools
import numpy as np
import matplotlib.pyplot as plt
from IPython import get_ipython
from IPython.display import display
from scipy.optimize import minimize
import astropy.units as u
from astropy.coordinates import Angle, SkyCoord
try:
    from google.colab import sheets  # will only work if you are running on Colab
except:
    pass

# web scraping / data access tools
import bs4 as bs
import urllib
import urllib.request
import pandas as pd
import s3fs

# parallel processing tools
from pathos.multiprocessing import ProcessingPool as Pool  # for multiprocessing inside Jupyter
import multiprocessing as mp

# microlensing tool
import MulensModel

import warnings

# Suppress *just that specific warning* MulensModelbecause we know what its doing and
# we want to keep the output clean
warnings.filterwarnings("ignore", message=".*does not have a limb-darkening coefficient.*")

## Single-Lens Fitting with `MulensModel`

In [ ]:
#@title Available finite source methods

finite_source_methods = [
    # Uniform source
    'finite_source_uniform_Gould94',               # 0, 10E-3 < rho < 1 (has a bug)
    'finite_source_uniform_Gould94_direct',        # 1, 10E-3 < rho < 1
    'finite_source_uniform_WittMao94',             # 2, rho < 0.01
    'finite_source_uniform_Lee09',                 # 3, rho > 0.01

    # Limb-darkened source
    'finite_source_LD_WittMao94',                  # 4, rho < 0.01
    'finite_source_LD_Yoo04',                      # 5, 10E-3 < rho < 1
    'finite_source_LD_Yoo04_direct',               # 6, 10E-3 < rho < 1
    'finite_source_LD_Lee09'                       # 7, rho > 0.01
]

Let's take a look at how different higher-order effects change the magnification model.  

In [ ]:
#@title Plotting the magnification models

# plot bounds
t_min = 2452750
t_max = 2452950
t_range = [t_min, t_max]

# Model parameters
t_0 =  2452848.06
u_0 = 0.133
t_E = 61.5
log_rho = -1.4 #@param {type:"slider", min:-3, max:0, step:0.1}
rho = 10**log_rho
pi_E_E = -0.5 #@param {"type":"slider","min":-5,"max":5,"step":0.1}
pi_E_N = 1.7 #@param {type:"slider", min:-5, max:5, step:0.1}
t_0_par = 2452848.06 # should not change during modelling and needs to be close to t_0

# Define a point source, point lens model
pspl = MulensModel.Model({'t_0': t_0, 'u_0': u_0, 't_E': t_E})

# Define a finite source, point lens model
fspl = MulensModel.Model({'t_0': t_0, 'u_0': u_0, 't_E': t_E, 'rho': rho})

# Define a parallax model
fspl_pllx = MulensModel.Model({'t_0': t_0,
                          'u_0': u_0,
                          't_E': t_E,
                          'rho': rho,
                          'pi_E_E': pi_E_E,
                          'pi_E_N': pi_E_N,
                          't_0_par': t_0_par  # fixed value for parallax calculations: ~t_0
                          },
                         ra='18:04:45.71',
                         dec='-26:59:15.2'
                         )

# Plot the magnification curve:
plt.close(0)
plt.figure(0)
pspl.plot_magnification(
    t_range=t_range,
    subtract_2450000=True,
    color='grey',
    linestyle='-',
    label='PSPL'
    )

# calculate the magnification curve using a finite source model
fspl.set_magnification_methods([2450000., finite_source_methods[1], 2470000.])  # rho = 0.1
fspl.plot_magnification(
    t_range=t_range,
    subtract_2450000=True,
    color='blue',
    linestyle='--',
    label='FSPL with uniform-brightness'
    )

# calculate the magnification curve using a finite source model with limb darkening
fspl.set_magnification_methods([2450000., finite_source_methods[5], 2470000.])  # rho = 0.1
fspl.plot_magnification(
    t_range=t_range,
    subtract_2450000=True,
    color='red',
    linestyle=':',
    linewidth=2,
    label='FSPL with Limb Darkening'
    )

# calculate the magnification curve using a finite source model and parallax
fspl_pllx.set_magnification_methods([2450000., finite_source_methods[1], 2470000.])  # rho = 0.1
fspl_pllx.plot_magnification(
    t_range=t_range,
    subtract_2450000=True,
    color='green',
    linestyle=':',
    linewidth=2,
    label='FSPL with parallax'
    )


# calculate the u_0 finite-source, parallax solution
fspl_pllx.set_magnification_methods([2450000., finite_source_methods[3], 2470000.])
parameters = ["t_0", "u_0", "t_E", "rho", "pi_E_E", "pi_E_N", "t_0_par"]
setattr(fspl_pllx.parameters, "u_0", -u_0)  # multiply u0 by -1
fspl_pllx.plot_magnification(
    t_range=t_range,
    subtract_2450000=True,
    color='purple',
    linestyle='--',
    linewidth=2,
    label=r'-$u_0$ FSPL with parallax'
    )

plt.legend(loc='upper left', bbox_to_anchor=(1.04, 1), borderaxespad=0)
plt.show()

If your version of `MulensModel` is working, this figure should have rendered without an error. If it is not working, ensure you have `mulensmodel>=3.4.0` installed.

> There are a few things to take away from this plot:
> * the finite source effect has a big effect on the shape of the magnification curve
> * the surface brightness model (e.g., uniform) for the source has much less of an effect
> * the degenerate parallax solutions may be noticably different with sufficiently large parallax
> * parallax does not need to be as big, for the affect to noticably change the magnification curve, compared with a static model.

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 1</h2>
    <p>Try playing with the parallax (<i>"pi_E_N"</i>, <i>"pi_E_E"</i>) and finite source (<i>"rho"</i>)parameters and see how they  change your magnification model.</p>
    <p><i>Note. This is not an interactive plot. You have to run the cell again after changing the values (or moving the slider, on Colab).</i></p>
    <br>
</div>

<a id="Data-Access"></a>
## OGLE EWS Bulk Light Curve Fit

Roman-style timelines include thousands of epochs, making model evaluations computationally heavy. Ground-based surveys like OGLE’s Early Warning System (EWS) provide faster alert metadata and estimated PSPL parameters. We will prototype our bulk fitter on these tractable datasets, then adapt it to simulated Roman seasons.


### Getting the Ground-Based Data


Let's start this process by scraping for some lightcurves and microlensing model parameter estimates.

In [ ]:
#@title Web scrapping functions

def get_data_url(event: str) -> str:
    '''Takes an event name and returns the URL for the data page.'''

    event = event.split('-') # split the event name into its components, seperated by '-'
    year = event[0]  # the first component is the year
    region = event[1].lower()  # the second component is region (e.g., blg or gd), which we need to make lower case.
    number = event[2]  #
    url = f'https://www.astrouw.edu.pl/ogle/ogle4/ews/{year}/{region}-{number}/phot.dat'

    return url

def fetch_event_data(url: str) -> pd.DataFrame:
    '''Takes a url and returns the data as a pandas dataframe.'''

    # Read the data from the URL
    response = urllib.request.urlopen(url)
    data = response.read().decode('utf-8')

    # Convert the data to a pandas DataFrame
    #df = pd.read_csv(StringIO(data), delim_whitespace=True, header=None, names=['HJD', 'I magnitude', 'magnitude error', 'seeing', 'sky level'])
    df = pd.read_csv(StringIO(data), sep=r'\s+', header=None, names=['HJD', 'I magnitude', 'magnitude error', 'seeing', 'sky level'])

    return df

# Test
event = '2017-BLG-0001'
event_data_url = get_data_url(event)
data = fetch_event_data(event_data_url)
print(data)

Great. Now that we can fetch individual light curves from the OGLE EWS website, the next ingredient is metadata. The alert table provides first-pass PSPL estimates that keep our fits stable and fast:

- `Tmax (HJD)` → proxy for $t_0$ (time of peak magnification)
- `Umin` → proxy for $u_0$
- `tau` → proxy for $t_E$
- `Event` → the human-readable identifier we use to reconstruct download URLs

We will shamelessly borrow these values as starting points so the optimizer begins near the global likelihood maximum instead of wasting time on divergent guesses.


In [ ]:
#@title More web scraping (this time for the EWS table)

def fetch_table_data(url):
    '''Takes a URL and returns the first table as a pandas DataFrame.'''
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source, 'lxml')
    table = soup.find_all('table')
    df = pd.read_html(StringIO(str(table)))[0]

    return df

ews_url = "https://ogle.astrouw.edu.pl/ogle4/ews/ews.html"  # https://ogle.astrouw.edu.pl/ogle4/ews/2025/ews.html for last year
ews_df = fetch_table_data(ews_url)
print(ews_df)

Next we attach a direct light-curve URL to every alert row so downstream code can iterate without re-deriving filenames. Think of this as building a lightweight catalog that couples OGLE metadata with downloadable photometry.

In [ ]:
#@title Adding a URL column to the data frame

# Add a new column to the EWS data frame ('ews_df'), using the column name 'event data url'.
ews_df['event data url'] = ews_df['Event'].apply(get_data_url)
print(ews_df)
print(min(ews_df['Tmax (HJD)']), max(ews_df['Tmax (HJD)']))
print(min(ews_df['Umin']), max(ews_df['Umin']))
print(min(ews_df['tau']), max(ews_df['tau']))

Nice! Now we have a miniature alert catalog—complete with URLs and first-pass PSPL estimates. Before we scale up, let’s sanity-check a single event by plotting the OGLE photometry alongside the EWS model parameters.

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 2</h2>
    <p>Extend the cell below by adding parallax parameters (<i>pi_E_E</i>, <i>pi_E_N</i>) and plotting the resulting model so you can see how higher-order terms perturb the fit.</p>
    <br>
</div>

In [ ]:
#@title Plotting the '2025-BLG-0001' event with the EWS model

###### [ ] add parallax to the model.

# Function to process a single event
def plot_event_data(i, ews_df):
    event = ews_df['Event'][i]
    print(event)
    print(ews_df.columns)
    url = ews_df['event data url'][i]
    data = fetch_event_data(url)
    t_0_0 = ews_df['Tmax (HJD)'][i]
    u_0_0 = ews_df['Umin'][i] * 1.1  # initial guess
    t_E_0 = ews_df['tau'][i] * 1.1  # initial guess
    rho_0 = 0.001  # initial guess
    ######
    # pi_E_E =
    # pi_E_N =
    # t_0_par =
    ######


    plt.close(i+1)
    plt.figure(i+1)

    plt.errorbar(data['HJD'],
                  data['I magnitude'],
                  yerr=data['magnitude error'],
                  fmt='x',
                  color='black'
                  )
    plt.axvline(ews_df['Tmax (HJD)'][i], color='blue', linestyle='--', label='EWS Tmax')

    plt.title(event)
    plt.xlabel('HJD')
    plt.ylabel('I magnitude')

    # Data as a list of numpy arrays
    data_list = [data['HJD'].to_numpy(), data['I magnitude'].to_numpy(), data['magnitude error'].to_numpy()]

    # Pack everything into MulensModel objects
    data_object = MulensModel.MulensData(data_list=data_list,
                                plot_properties={'color': 'thistle',
                                                 'label': 'OGLE',
                                                 'marker': 'x',
                                                 'markersize': 2
                                                 },
                                phot_fmt='mag',
                                bandpass='I'
                                )
    ######
    fspl_model = MulensModel.Model({'t_0': t_0_0,
                                    'u_0': u_0_0,
                                    't_E': t_E_0,
                                    'rho': rho_0}
                                   )  # add parallax parameters
                                      # (initialize with (0.0, 0.0))
    ######
    fspl_model.set_magnification_methods([t_0_0 - 3.0 * t_E_0,
                                          finite_source_methods[0],
                                          t_0_0 + 3.0 * t_E_0
                                          ],
                                          source=None
                                          )  # rho <= 0.1
    event_object = MulensModel.Event(datasets=data_object, model=fspl_model)

    ######
    parameters_to_fit = ["t_0", "u_0", "t_E", "rho"]  # add parallax parameters

    ######

    # Plot the initial model
    ######
    label = 'EWS model: %1.3f, %1.3f, %1.3f, %1.3f' %(t_0_0,
                                                      u_0_0,
                                                      t_E_0,
                                                      rho_0
                                                      )  # Add parallax to the
                                                         # plot label
    ######
    event_object.plot_model(color='r',
                            linestyle=':',
                            t_range=[min(data['HJD']),
                                     max(data['HJD'])
                                    ],
                            label=label
                            )

    plt.legend()
    plt.savefig(f'./{event}.png', bbox_inches='tight')
    plt.show()

plot_event_data(0, ews_df)


Is it working?

If so, we can move on and test our fitting algorithms.

### Fitting a Basic PSPL Model


First, we’ll need an **objective function** — a way to measure how well our model fits the data (or, more precisely, how likely the model is to have generated the data, assuming Gaussian noise).

In [ ]:
#@title Objective function
def mulens_neglogP_function(theta, parameters_to_fit, event, verbose=False):
    ''' negative log prob function for MulensModel fitting '''

    # Create a dictionary from theta values for easier access
    params = dict(zip(parameters_to_fit, theta))

    # unpack params
    t_0_value = params['t_0']
    u_0_value = params['u_0']
    t_E_value = params['t_E']
    # rho is handled later

    # Prior Checks
    if not ((2460600) <= t_0_value <= (2461000)):  # this needs to change if you
                                                   # use this code to fit a different
                                                   # season
                                                   # For example, for exercies X
                                                   # !!!
        return np.inf
    if not (0.000001 <= u_0_value <= 2.5):  # u_0 can't be 0
        return np.inf
    if not (0.1 <= t_E_value <= 700):
        return np.inf

    # Handle rho or log_rho prior
    if 'rho' in params:
        rho_value = params['rho']
        if not (0 <= rho_value <= 0.2): # Assuming rho >= 0 is desired
            return np.inf
    elif 'log_rho' in params:
        log_rho_value = params['log_rho']
        # Check log_rho lower bound first
        if log_rho_value < -10: # Check the log value directly
             return np.inf
        rho_value = 10**log_rho_value # Convert to rho for the upper bound check
        if rho_value > 0.2: # Check rho upper bound
             return np.inf

    # Update Model Parameters
    # If all prior checks passed, NOW update the model
    setattr(event.model.parameters, 't_0', t_0_value)
    setattr(event.model.parameters, 'u_0', u_0_value)
    setattr(event.model.parameters, 't_E', t_E_value)
    setattr(event.model.parameters, 'rho', rho_value)
    # Add other parameters if needed

    # Example flux priors
    penalty = 0.0
    '''
    dataset = event.datasets[0]
    event.fit_fluxes() # This needs the model parameters to be set correctly
    ([FS], FB) = event.get_flux_for_dataset(dataset)

    if verbose:
        print(f'FS: {FS}, FB: {FB}')

    # Flux priors (check AFTER fitting fluxes)
    if FB <= 0:
        penalty = ((FB / 100)**2) # why 100? I told you, vibes.
    if FS <= 0 or (FS + FB) <= 0:
        return np.inf # Return inf if fluxes are non-physical '''

    # Calculate Chi2
    chi2 = event.get_chi2()
    if verbose:
        print('chi2 = ', chi2)

    # Return the objective function value (negative log likelihood ~ chi2/2)
    # Scipy minimize finds the minimum, so we return chi2 (or chi2/2)
    return chi2/2.0 + penalty # Technically negLogP is chi2/2 + constant

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 3</h2>
    <p>Test out a fit of a subsample of the EWS data, by running the following 2 cells.</p>
    <br>
</div>

In [ ]:
#@title Function to process a single event
def process_event(i, ews_df, n, start_time, verbose=True, log_rho_prior=True):

    # Event stuff
    event = ews_df['Event'][i]
    event_start_time = time.time()
    url = ews_df['event data url'][i]
    data = fetch_event_data(url)

    # Model stuff
    t_0_0 = ews_df['Tmax (HJD)'][i]
    # We are going to change the next guesses from those provided by the EWS, to
    # give our optiizer some room to move
    u_0_0 = ews_df['Umin'][i] * 1.1  # initial guess
    t_E_0 = ews_df['tau'][i] * 1.1  # initial guess
    rho_0 = 0.001  # initial guess

    # Crop the data so that we don't have to fit 10 years worth
    t_window = (t_0_0 - 10.0 * t_E_0, t_0_0 + 10.0 * t_E_0)
    t_window = (max(min(data['HJD']), t_window[0]), min(max(data['HJD']), t_window[1]))
    # This creates a boolean mask: True for rows within the window, False otherwise
    time_mask = (data['HJD'] >= t_window[0]) & (data['HJD'] <= t_window[1])
    # Apply the mask to filter the DataFrame
    data = data[time_mask]

    if t_E_0 > 50:  # liekly remnant (they do derpy stuff in the wings)
        mag_method = finite_source_methods[0]  # Note: `finite_source_uniform_Gould94`
                                               # is not ideal for events where
                                               # rho ≲ 1e-3, but we use it here
                                               # to ensure fast and consistent
                                               # modeling while testing functions.
                                               # For high-mass lenses or precision
                                               # modeling, consider switching to
                                               # `WittMao94` variants (method 2
                                               # or 4).
    else:  # likely MS star
        mag_method = finite_source_methods[0]

    if verbose:
        print('\n', event)
        print('-----------------')
        print(f't_0_0: {t_0_0:1.3f}')
        print(f'u_0_0: {u_0_0:1.3f}')
        print(f't_E_0: {t_E_0:1.3f}')
        print(f'rho_0: {rho_0:1.5f}')
        print(f'Method = {mag_method}')
        print('-----------------')
        print(f'Time started: {time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())}')

    if i % n == 0:
        plt.close(i)
        plt.figure(i)

        plt.errorbar(data['HJD'],
                     data['I magnitude'],
                     yerr=data['magnitude error'],
                     fmt='x',
                     color='black'
                     )
        plt.axvline(ews_df['Tmax (HJD)'][i], color='blue', linestyle='--', label=r'EWS $T_{max}$')

        plt.title(event)

    # Data as a list of numpy arrays
    data_list = [data['HJD'].to_numpy(), data['I magnitude'].to_numpy(), data['magnitude error'].to_numpy()]

    # Pack everything into MulensModel objects
    data_object = MulensModel.MulensData(data_list=data_list,
                                plot_properties={'color': 'thistle',
                                                 'label': 'OGLE',
                                                 'marker': 'x',
                                                 'markersize': 2
                                                 },
                                phot_fmt='mag',
                                bandpass='I'
                                )
    fspl_model = MulensModel.Model({'t_0': t_0_0, 'u_0': u_0_0, 't_E': t_E_0, 'rho': rho_0})

    fspl_model.set_magnification_methods([t_window[0],
                                          mag_method,
                                          t_window[1]
                                          ],
                                          source=None
                                          )  # rho <= 0.1
    event_object = MulensModel.Event(datasets=data_object, model=fspl_model)

    # Plot the initial model
    if i % n == 0:
        event_object.plot_model(color='r',
                                linestyle=':',
                                t_range=[min(data['HJD']),
                                         max(data['HJD'])
                                         ],
                                label='Initial Guess: %1.3f, %1.3f, %1.3f, %1.5f' %(t_0_0, u_0_0, t_E_0, rho_0)
                                )

    # get initial chi2
    initial_chi2 = event_object.get_chi2()

    # index order
    parameters_to_fit = ["t_0", "u_0", "t_E", "rho"]
    if log_rho_prior:
        parameters_to_fit = ["t_0", "u_0", "t_E", "log_rho"]

    # Fit using scipy Nelder-Mead
    if log_rho_prior:
        result = minimize(mulens_neglogP_function,
                          [t_0_0, u_0_0, t_E_0, np.log10(rho_0)],
                          args=(parameters_to_fit,
                          event_object),
                          method='Nelder-Mead'
                          ) # log rho prior
    else:
        result = minimize(mulens_neglogP_function,
                          [t_0_0, u_0_0, t_E_0, rho_0],
                          args=(parameters_to_fit,
                          event_object),
                          method='Nelder-Mead'
                          )

    # Make sure the values in event_object are the best fit values
    neglogP_final = mulens_neglogP_function(result.x, parameters_to_fit, event_object, verbose=False)

    if verbose:
        print('-----------------')
        print(f'Time elapsed: {time.time() - event_start_time} seconds')
        print('-----------------')
        neglogP_final = mulens_neglogP_function(result.x, parameters_to_fit, event_object, verbose=True)
        print('-----------------')
        print(f't_0: {event_object.model.parameters.t_0:1.3f} ({result.x[0]:1.3f})')
        print(f'u_0: {event_object.model.parameters.u_0:1.3f} ({result.x[1]:1.3f})')
        print(f't_E: {event_object.model.parameters.t_E:1.3f} ({result.x[2]:1.3f})')
        print(f'rho: {event_object.model.parameters.rho:1.5f} ({result.x[3]:1.5f})') # Always rho here
        print('-----------------')
        print(f'Final -logP check: {neglogP_final}')
        print('-----------------')
        print(f'Initial chi2 (from initial guess): {initial_chi2}')
        print(f'Final chi2 (from event object): {event_object.get_chi2()}')
        print(f'Delta chi2: {event_object.get_chi2() - initial_chi2}')
        print('Delta chi2/dof: ',(event_object.get_chi2() - initial_chi2) / (len(data['HJD']) - 4))
        print('-----------------')


    # Plot the fit model and show (event_object now has correct params)
    if i % n == 0:
        # Construct the label string from the corrected event_object model state
        label = '''Best Fit: {0.t_0:1.3f}, {0.u_0:1.3f}, {0.t_E:1.3f},
                   {0.rho:1.5f}'''.format(event_object.model.parameters)

        event_object.plot_model(color='r',
                                linestyle='-',
                                t_range=[min(data['HJD']), max(data['HJD'])],
                                label=label
                                )
        # Use parameters from the event object model for plot limits etc.
        t_0 = event_object.model.parameters.t_0
        t_E = event_object.model.parameters.t_E
        # Define plot window based on final params or keep original? Your choice.
        # Using original t_window:
        plt.xlim(max(min(data['HJD']), t_window[0]),
                 min(max(data['HJD']), t_window[1])
                 )
        plt.legend(loc='upper left')
        plt.xlabel('HJD')
        plt.ylabel('I magnitude')
        plt.savefig(f'./{event}.png', bbox_inches='tight')
        plt.show()

    # Return the actual best-fit parameters found by minimize
    # You might want to return the converted values for consistency
    final_params_values = list(result.x)
    if log_rho_prior:
        log_rho_idx = parameters_to_fit.index('log_rho')
        final_params_values[log_rho_idx] = 10**final_params_values[log_rho_idx] # Convert log_rho back to rho

    event_object.fit_fluxes() # This needs the model parameters to be set correctly
    ([FS], FB) = event_object.get_flux_for_dataset(event_object.datasets[0])
    if verbose:
        print(f'FS: {FS}, FB: {FB}')
        print('-----------------')
    final_params_values = [FS, FB] + final_params_values

    return i, final_params_values # Return values, maybe with rho always as rho

In [ ]:
#@title Test fitting

# numpy array for the fit params
fit_params = np.zeros((ews_df.shape[0], 8))

plot_fraction = 0.1  # fraction of events to show plots for
N = 10  # number of events to fit

start_time = time.time()
for i in range(N):
    i, result = process_event(i,
                              ews_df,
                              int(1/plot_fraction),
                              start_time,
                              verbose=True,
                              log_rho_prior=True
                              )
    n = len(result)
    fit_params[i, :n] = result

# Geez, how long is this going to take?
time_at_N = time.time()
time_for_N = time_at_N - start_time
print(f'Time taken to fit the first {N}: {time_for_N} seconds')

# Estimate completion time
completion_time = time_at_N + (ews_df.shape[0] - N) / N * time_for_N
print(time_at_N, (ews_df.shape[0] - N) / N, time_for_N)

# Print the completion time in human readable format hr:min:sec
print(f'Estimated season completion time: {time.strftime("%H:%M:%S", time.localtime(completion_time))}')
end_time = time.time()

After fitting a subsample, inspect the posterior summaries. Do the recovered $t_0$, $u_0$, and $t_E$ distributions cluster around the OGLE seeds? Large skews or multimodal shapes often signal that priors or initial guesses need another pass.

In [ ]:
#@title Exercise 3 fit parameter distributions

def plot_histograms(fit_params, exercise):
    ''' '''

    plt.close(100 + exercise)
    plt.figure(100 + exercise)
    fig, axes = plt.subplots(8, 1, figsize=(5, 25))

    # FS values
    min_FS = min(fit_params[:, 0])
    max_FS = max(fit_params[:, 0])
    print(min_FS, max_FS)
    axes[0].hist(fit_params[:, 0], bins=20, color='#361d49')
    axes[0].set_xlabel(r'$F_\text{S}$')
    axes[0].set_ylabel('Frequency')
    #axes[0].set_yscale('log')

    # FB values
    min_FB = min(fit_params[:, 1])
    max_FB = max(fit_params[:, 1])
    print(min_FB, max_FB)
    axes[1].hist(fit_params[:, 1], bins=20, color='#361d49')
    axes[1].set_xlabel(r'$F_\text{B}$')
    axes[1].set_ylabel('Frequency')
    #axes[1].set_yscale('log')

    # t_0 values
    min_t0 = min(fit_params[:, 2])
    max_t0 = max(fit_params[:, 2])
    print(min_t0, max_t0)
    axes[2].hist(fit_params[:, 2], bins=20, color='#361d49')
    axes[2].set_xlabel(r'$t_0$')
    axes[2].set_ylabel('Frequency')
    #log the y axis
    #axes[2].set_yscale('log')

    # u0 values
    min_u0 = min(fit_params[:, 3])
    max_u0 = max(fit_params[:, 3])
    print(min_u0, max_u0)
    axes[3].hist(fit_params[:, 3], bins=20, color='#361d49')
    axes[3].set_xlabel(r'$u_0$')
    axes[3].set_ylabel('Frequency')
    #axes[3].set_yscale('log')

    # tE values
    min_tE = min(fit_params[:, 4])
    max_tE = max(fit_params[:, 4])
    print(min_tE, max_tE)
    axes[4].hist(fit_params[:, 4], bins=20, color='#361d49')
    axes[4].set_xlabel(r'$t_\text{E}$')
    axes[4].set_ylabel('Frequency')
    #axes[4].set_yscale('log')

    # rho values
    min_rho = min(fit_params[:, 5])
    max_rho = max(fit_params[:, 5])
    print(min_rho, max_rho)
    print(np.log10(min_rho), np.log10(max_rho))
    axes[5].hist(np.log10(fit_params[:, 5]), bins=20, color='#361d49')
    axes[5].set_xlabel(r'$\rho$')
    axes[5].set_ylabel('Frequency')
    #axes[5].set_yscale('log')

    # pi_E_E values
    min_FS = min(fit_params[:, 6])
    max_FS = max(fit_params[:, 6])
    print(min_FS, max_FS)
    axes[6].hist(fit_params[:, 6], bins=20, color='#361d49')
    axes[6].set_xlabel(r'$\pi_{\text{E},E}$')
    axes[6].set_ylabel('Frequency')
    #axes[6].set_yscale('log')

    # pi_E_N values
    min_FB = min(fit_params[:, 7])
    max_FB = max(fit_params[:, 7])
    print(min_FB, max_FB)
    axes[7].hist(fit_params[:, 7], bins=20, color='#361d49')
    axes[7].set_xlabel(r'$\pi_{\text{E},N}$')
    axes[7].set_ylabel('Frequency')
    #axes[7].set_yscale('log')

    plt.show()

fit1_params = fit_params[:N].copy()

plot_histograms(fit1_params, 3)

### Priors


We will now interpret the deeply non-physical result where the background flux is very negative.

Likely at least a few of your events had negative $F_{\textrm{B}}$ (*we're looking at you, 2025-BLG-0030*). This is a pretty common issue with single-lens modelling.

But what does it mean?

Nothing that makes any physical sense. If the blend is a little bit negative, that can be explained by systematics in the photometry (e.g. the background was measured with very faint star in it, so our flux scale was zeroed slightly wrong), but there is no reason for the blend to be very negative; we can't detect anti-photons.

So, how can we punish the optimizer for its sins? It's actually pretty simple. We punish it by adding a penalty to the objective function. If the penalty is too abrupt, it can sometimes interfere with a gradient descent optimizer's ability to calculate gradients. But if we put a gradual penalty on it, the penalty acts to lead the optimizer in the right direction. The exact shape of the penalty doesn't really matter, it just needs to make more sense than giant negative fluxes.

This kind of penalty behaviour is called a prior. Where we use prior knowledge to inform the most probability landscape. Basically we tell optimization, “a very negative blend is dumb - don't do that.”

You might argue that you would prefer to approach your modelling from an agnostic perspective and I appreciate your integrity. The problem with that argument though is the assumption that no prior is agnostic. Because no prior is like telling your optimizer, "negative blend is just as reasonable as positive blend." That's not agnostic. You've still informed the fit. You just informed it that every solution was equally possible, which you know it's not. This not-actually-agnostic prior, where you don't code in any sort of penalties, is called a uniform prior. The concept of "no prior" is a fallacy.

So let's put a reasonable prior on the blend flux so that it stops acting up. Our other fit parameters can continue to have truncated priors (bounds).

The next question you might ask is, "how do I know which prior is the right prior" the answer to that is: vibes.
I'm not even joking. You choose a prior that is physically informed or informed by your "prior knowledge" of what your solution should be and don't worry overly much about it. You should however, always be wary that your priors are not so strong that they dominate the fit. If what you get out from a fit is very similar to your prior, that is an indication that you have been too heavy handed and you need to loosen the leash.

For example, if you made your prior a hard bound, requiring $F_{\textrm{B}}>0$, and your best fit solution was $F_{\textrm{B}}\approx0$, you have likely stopped the optimizer from exploring valid parameter space.

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 4</h2>
    <p>Edit <i>mulens_neglogP_function()</i> to use a Gaussian prior to constrain the blend flux, if it is below 0; use a piecewise prior combining:</p>
    <ul>
        <li>A uniform prior for <i>F</i><sub>B</sub><i>>0</i></li>
        <li>A Gaussian penalty for <i>F</i><sub>B</sub><i><0</i></li>
    </ul>
    <br>
</div>

In [ ]:
#@title Exercise 4 fit parameter distributions

fit2_params = fit_params[:N].copy()
plot_histograms(fit2_params, 4)

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 5</h2>
    <p>If everything is working, try adding parallax to model in the EWS event processing function.</p><br>
</div>

In [ ]:
#@title Exercise 5 fit parameter distributions

fit3_params = fit_params[:N].copy()
plot_histograms(fit3_params, 5)

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 6</h2>
    <p>Increase the number of events you are testing on (<i>N</i>) so that we can do some more robust benchmarking. You should aim for a number of events that means the above cells takes about 5 minutes to run.</p><br>
</div>

In [ ]:
#@title Exercise 6 fit parameter distributions

fit4_params = fit_params[:N].copy()
plot_histograms(fit4_params, 6)

### Parallel Processing


We are going to demonstrate speeding this process up with parallel processing. However, there are a few things you should know first.

We're going to demonstrate how to speed up batch fitting using parallel processing. But before we dive in, there are a few things you should know.

If you're running this notebook on Google Colab, your code is executing on a cloud-based virtual machine. Colab typically allocates only two CPU cores per session. While we can parallelize across these, the speed-up won't be dramatic.

If you have Jupyter and Python installed on your local machine, you can run this notebook there to take advantage of your full hardware.

Want to keep the Colab interface but use your local computer's power?
You can connect to a local runtime by following [these instructions](https://research.google.com/colaboratory/local-runtimes.html).
This allows Colab to use your local machine's resources while maintaining the notebook interface.

⚠️ If you choose this route, your notebook will use local packages. You should install the required environment using the provided `.yml` file and `conda` (or your preferred environment manager). See the setup instructions at the start of this notebook.

This notebook uses the `pathos` package for parallelization.
In scripts, you'll more commonly see `multiprocessing` used—it has a very similar interface. However, multiprocessing has known issues inside Jupyter notebooks, which is why we're using pathos instead.

Pathos is great for educational work in notebooks, but it has quirks. For example:
* It caches the function you give it the first time you run it.
* If you change that function afterward, it won't update unless you restart the kernel.

That's why we finalized our `fit_event()` function earlier—so we could parallelize it now with confidence.

In general, we don't recommend using all available cores for parallelization:

```python
with Pool(processes=mp.cpu_count()) as pool:
```

This often results in slower execution, as the system spends time managing threads instead of doing actual computation. It's usually better to leave **one core free** for system processes:

```python
with Pool(processes=mp.cpu_count() - 1) as pool:
```

That said, Colab's VM only provides two cores, so `mp.cpu_count() - 1`  is pointless here. For now, let's just try a pathos batch run on 2 cores and compare the timing to the serial loop from earlier.

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 7</h2>
    <p>Test how different values of processes in the <i>Pool()</i> function affect your batch fitting time.</p>
    <p>Try using 1, 2, and maybe <i>mp.cpu_count()</i>` if you're on your local runtime.</p>
    <p>What's the fastest? What's the most efficient?</p><br>
</div>

In [ ]:
# Main function to parallelize the processing
def run_parallel_processing(plot_fraction=None, kill_after=None):
    # numpy array for the fit params
    fit_params = np.zeros((ews_df.shape[0], 8))

    if plot_fraction is None:
        plot_fraction = 0.1
    if kill_after is None:
        kill_after = 30

    start_time = time.time()

    # Create a pool of worker processes
    print('CPU count:', mp.cpu_count())
    with Pool(processes=mp.cpu_count()) as pool: # if running locally, keep the
                                                 # process count < mp.cpu_count()
                                                 # or it will be very slow
        results = pool.map(lambda i: process_event(i, ews_df, int(1/plot_fraction), start_time), range(kill_after))
        for i, params in results:
            n = len(params)
            fit_params[i, :n] = params

    print("Total time:", time.time() - start_time)

    # Save fit_params if needed
    # np.save('fit_params.npy', fit_params)

    return fit_params

# Run the parallel processing function
######
#fit_params = run_parallel_processing() # test run
fit_params = run_parallel_processing(kill_after=N, plot_fraction=plot_fraction) # test run
#fit_params = run_parallel_processing(kill_after=ews_df.shape[0],
#                                     plot_fraction=0.01
#                                     ) # full run
######

In [ ]:
fit_params[:N]

In [ ]:
#@title Exercise 7 fit parameter distributions

fit5_params = fit_params[:N].copy()
plot_histograms(fit5_params, 7)

In beta testing, the timing results for fitting 30 events were:

| Processing | Cores | Events (N) | Plots | Time |
| :-: | :-: | :-: | :-: | :-: |
| Serial | 1 | 30 | 100% | 246s |
| Serial | 1 | 30 | 10% | 237s |
| Parallel | 2 | 30 | 100% | 218s |
| Parallel | 2 | 30 | 10% | 213s |

A modest improvement, considering the extra coding overhead. However, on a local machine with more cores:

| Processing | Cores | Events (N) | Plots | Time |
| :-: | :-: | :-: | :-: | :-: |
| Serial | 1 | 30 | 100% | 150s |
| Serial | 1 | 30 | 10% | 144s |
| Parallel | 7 | 30 | 100% | 56s |
| Parallel | 7 | 30 | 10% |	55s |


This shows the real power of parallelisation—when the hardware can actually support it.

If you even encounter this kind of problem in the wild, where you a very parallelisable job with limited time to implement parallelization, it's always worth considering poor-man parallelisation:
Break the job into batches, run multiple scripts or terminals at once, and let your operating system juggle the rest.

If your code is loop heavy, look for vectorisation opportunities using `numpy`.
Or `cython`, if that's not an option.

Now that you've seen how parallelisation affects efficiency, you're ready to run a full-season fit.
However, if you're short on time, it's entirely reasonable to skip this exercise.

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 8</h2>
    <p>Use the more efficient method (serial or parallel) to process the full season of EWS data.</p>
    <p>Plot histograms of the resulting parameter distributions (e.g., <i>t</i><sub>E</sub>, <i>u</i><sub>0</sub>, etc.).</p>
    <p>Look for trends and outliers.</p><br>
</div>

In [ ]:
#@title Exercise 8 fit parameter distributions

fit6_params = fit_params[:N].copy()
plot_histograms(fit6_params, 8)

### Custom Event Finder


If we weren't borrowing initial fit parameters from the OGLE EWS, these fits would fail completely under downhill optimizers. Starting too far from the truth in parameter space means the optimizer gets trapped in local minima - the peaks and valleys of the likelihood landscape become a cage, not a guide.

We'll explore more robust (and computationally expensive) methods in the next section. But even with advanced samplers, having a good initial guess makes the entire process faster, more stable, and more scientifically honest.

Different ground-based surveys use different strategies to identify microlensing events in stellar light curves. You, however, have a luxury they don't: you already know which light curves contain events. That means your challenge isn't classification—it's localization.

And the most critical parameter to localize is $t_0$.  
Other parameters (like $u_0$, $t_\textrm{E}$, and $\rho$) can be guessed with population-based heuristics. But if your $t_0$ is wrong, your fit will miss the peak entirely - and what you get back will be noise-dressed nonsense.

This is a central problem in bulk fitting single-lens microlensing events:  
> We need an event-finding algorithm to guess $t_0$ accurately.

You now have:
- A full season of OGLE data
- Light curves that are guaranteed to contain events
- A set of academic papers describing common event-finding strategies:
  - [OGLE Early Warnign System (EWS)](https://ui.adsabs.harvard.edu/abs/1994AcA....44..227U/abstract)
  - [KMTNet Alert Finder](https://ui.adsabs.harvard.edu/abs/2018arXiv180607545K/abstract)
  - [KMTNet Event Finder](https://ui.adsabs.harvard.edu/abs/2018AJ....155...76K/abstract)
  - [KMTNet Anomaly Finder](https://ui.adsabs.harvard.edu/abs/2021AJ....162..163Z/abstract)
  - [RTModel (Real-Time Modeling)](https://ui.adsabs.harvard.edu/abs/2024A%26A...688A..83B/abstract) (see their section 4)

Do you think you can do better?

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 9</h2>
    <p>Write an algorithm to estimate $t_0$ for microlensing events. Keep it simple. Keep it efficient. The rest of your fit may depend on it.</p><br>
</div>

In [ ]:
#@title Your code goes here

### Advanced Modeling Techniques and Higher-Order Models


The idea that a good $t_0$ guess is enough for a good fit holds true for FSPL events. But when an FSPL model fails to fit cleanly, that's often a sign that higher-order effects are at play. You've encountered some of these in other chapters of this course.

We know that binary stars are common in the galaxy. And yet, we often model events using PSPL or FSPL, as if single-lens-single-source events are the default. In fact, recent simulations of ground-based surveys using modern Galactic models suggest that around 50% of single-peaked microlensing events are actually "hidden binaries"—either:
- **Binary lenses** (multiple objects in the lens system), or  
- **Binary sources** (two source stars lensed simultaneously).

#### Binary Source Stars 

They introduce subtle distortions to the light curve and can easily masquerade as single-lens, binary-lens events.  

If you'd like to learn more, see this [notebook on binary sources](https://github.com/AmberLee2427/TheMicrolensersGuideToTheGalaxy/blob/main/Notebooks/BinarySource.ipynb).

#### Binary Lenses

These are a different beast entirely. They require:
- Higher-order models
- Optimizers that can escape local minima (e.g. **Monte Carlo** methods that allow uphill steps)

  ([This notebook](https://github.com/AmberLee2427/TheMicrolensersGuideToTheGalaxy/blob/main/Notebooks/Modelling.ipynb) on modelling methods commonly used in microlensing may also be of interest)
- Often a **grid search** in $s$, $q$, and $\rho$ to even get reasonable initial conditions

Even with that, **degeneracies are common** — and easy to miss. For example, see the modeling of OGLE-2016-BLG-1195 (Shartzvald et al., 2017; Bond et al., 2017; Gould et al., 2023; Vandorou et al., 2024), where a viable degenerate solutions were overlooked at each stage of modelling.

If a light curve shows dramatic multi-peaked deviations from a Paczyński shape, you're **almost certainly dealing with lens multiplicity** (unless some other astrophysical event has contaminated your lightcurve). But determining the *correct* model requires balancing:
- Evidence for complexity, and  
- The principle of parsimony (Occam's Razor), while knowing full well that these “complex” arrangements are not **rare** in *occurance* so much as in *observability*.

---

For more on binary lense microlensing fitting, see the next notebook in this series: [Fitting Binary Lenses](../Session%20B:%20Single%20Lens%20&%20Pipelines/Single_Lens_Pipeline.ipynb) (also available on [GitHub](https://github.com/rges-pit/data-challenge-notebooks/blob/main/AAS%20Workshop/Session%20B%3A%20Single%20Lens%20%26%20Pipeline/Single_Lens_Pipeline.ipynb) and [Colab]())

---

#### Other higher-order effects not covered in this series (but worth knowing):
- **Lens orbital motion**  
- **Xallarap** (source orbital motion)  
- **Multiple lenses** (e.g. triple lenses)  
- **Variable stars** (source, lens, or blend stars with intrinsic variability)  
- Variable **blending** (ambient stars moving in/out of the aperture/PSF)  
- General data **systematics**  



<a id="Pipeline-Automation"></a>
## Full Season Roman Fit

First, the data. There is a trove full of "simple" *Roman*-like microlensing light curves from the 2018 WFIRST Data Challenge. We start our mini data challenge by cloning that and pulling out all the relevant light curves.

In [ ]:
# Assume local data path
DATA_ROOT = Path('data-challenge-1')
LOCAL_MASTER = DATA_ROOT / 'Answers/master_file.txt'
LOCAL_HEADER = DATA_ROOT / 'Answers/wfirstColumnNumbers.txt'
LOCAL_EVENT_INFO = DATA_ROOT / 'event_info.txt'
LOCAL_LC_DIR = DATA_ROOT / 'lc'

# Assume Nexus data path
TIER = "2018-test"  # workshop tier mirrored on Nexus
S3_BASE = f"s3://rmdc26-data-public/{TIER}"
S3_MASTER = f"{S3_BASE}/master_file.txt"
S3_HEADER = f"{S3_BASE}/wfirstColumnNumbers.txt"
S3_EVENT_INFO = f"{S3_BASE}/event_info.txt"
S3_LC_TEMPLATE = f"{S3_BASE}/{{event_id}}.dat"

### Nexus Data Access
If you are running on the Roman Research Nexus, the `data-challenge-1/` directory used by this notebook is mirrored at `s3://rmdc26-data-public/2018-test/`. That bucket contains everything a local clone would provide:

- `master_file.txt` – event catalog with 96 metadata columns
- `wfirstColumnNumbers.txt` – column descriptions for the master file
- `event_info.txt` – sky coordinates, extinction, and distance estimates
- `ulwdc1_XXX.dat` – combined W149/Z087 light curves for each event

The cell below shows how to stream a single event directly from S3 and plot the photometry per band.



In [ ]:
# NEXUS-ONLY
EVENT_ID = "ulwdc1_001"  # update as needed during the workshop
EVENT_URI = S3_LC_TEMPLATE.format(event_id=EVENT_ID)

print(f"Streaming {EVENT_URI} ...")
with open_source(EVENT_URI, 'r') as handle:
    s3_data = np.genfromtxt(handle, names=True, dtype=None, encoding=None)

bands = np.unique(s3_data['band'])
plt.figure(figsize=(7, 4))
for band in bands:
    mask = s3_data['band'] == band
    plt.errorbar(
        s3_data['HJD'][mask], s3_data['mag'][mask], yerr=s3_data['mag_err'][mask],
        fmt='.', label=f'Band {band}', alpha=0.75
    )

plt.gca().invert_yaxis()
plt.title(f"{EVENT_ID} (S3 stream)")
plt.xlabel("HJD")
plt.ylabel("Magnitude")
plt.legend(title="Band")
plt.show()


### Alternative Data Access

If you are running the notebook outside of the Nexus, you can acces the data for out next project by clones the 2018 data challenge repo. Otherwise, you do not need to run the cell below.

In [ ]:
#@title Cloning the GitHub repository (local/Colab)
from pathlib import Path

DATA_ROOT = Path('data-challenge-1')
if not DATA_ROOT.exists():
    !git clone https://github.com/microlensing-data-challenge/data-challenge-1.git
    !tar -xzvf data-challenge-1/lc.tar.gz -C data-challenge-1/
else:
    print("data-challenge-1 already present; skipping clone/untar.")

### Load and Sort the Data

Now that we have a means to access the data, no matter our chosen platform, we need to tidy it into a more programatically friendly form.

In [ ]:
# Read data into memory, agnostic of the data source

def prefer_local(local_path: Path, s3_path: str) -> str:
    """Return the local path if it exists, otherwise fall back to the S3 URI."""
    return str(local_path) if local_path.exists() else s3_path

MASTER_SOURCE = prefer_local(LOCAL_MASTER, S3_MASTER)
HEADER_SOURCE = prefer_local(LOCAL_HEADER, S3_HEADER)
EVENT_INFO_SOURCE = prefer_local(LOCAL_EVENT_INFO, S3_EVENT_INFO)


def open_source(path_or_uri: str, mode: str = 'r'):
    """Open a local file or S3 URI with a unified interface."""
    path_or_uri = str(path_or_uri)
    if path_or_uri.startswith('s3://'):
        return s3fs.S3FileSystem(anon=True).open(path_or_uri, mode)
    return open(path_or_uri, mode)


def read_table(path_or_uri: str, **kwargs):
    """Read a whitespace-delimited table from local disk or S3 into a DataFrame."""
    with open_source(path_or_uri) as handle:
        return pd.read_csv(handle, **kwargs)

Feel free to `Shift` + `Enter` through the rest of this section until you reach **Adjusting the Model for an L2 Orbit**, if you just want the pipeline results.

This dataset includes 293 lightcurve, 74 of which are single lens events. We can cheat a little and specifically pull out the events that we know to be single lenses, keeping the challenge tractable for completion within the hour, with the added benefit of making the strangley organized `master_file.txt` easier to wrangle.

In [ ]:
# Derive descriptive column names from the header file (local path or S3)
# The wfirstColumnNumbers.txt file is tab-separated with columns:
# colno, name, units, min, max, bins, lgscl, label, xmod

header_df = read_table(
    HEADER_SOURCE,
    sep=r'\s+',
    comment='#',
    header=None,
    engine='python',
    dtype=str  # IMPORTANT: keep everything as string initially
)

# The first column is the index, the second is the name
# Extract just these two columns
header_df = header_df.iloc[:, [0, 1]]
header_df.columns = ['index', 'name']

# Convert index to numeric, dropping any rows that fail
header_df['index'] = pd.to_numeric(header_df['index'], errors='coerce')
header_df = header_df.dropna(subset=['index'])
header_df['index'] = header_df['index'].astype(int)

# Start with placeholder names
colnames_96 = [f'col_{i:02d}' for i in range(96)]

# Map column indices to their names from the header file
for idx, name in zip(header_df['index'], header_df['name']):
    if 0 <= idx < len(colnames_96):
        # Skip separator rows marked with '|'
        if name != '|':
            colnames_96[idx] = name

# Override specific columns that we reference later with consistent names
# These ensure compatibility with the L2 orbit code
colnames_96[30] = 'u0'           # impact parameter
colnames_96[32] = 't0'           # time of closest approach  
colnames_96[33] = 'tE'           # Einstein crossing time
colnames_96[36] = 'piE'          # parallax amplitude
colnames_96[37] = 'rhos'         # source size (rho_star)
colnames_96[73] = 'N'            # number of data points (or chi2_0)
colnames_96[75] = 'Delta chi2'   # delta chi-squared
colnames_96[92] = 'sim type'     # simulation type (dcnormffp, ombin, etc.)
colnames_96[94] = 'filename'     # filename string
colnames_96[95] = 'lc_number'    # light curve number

# Replace any remaining '|' separators or empty strings with placeholder names
colnames_96 = [
    name if isinstance(name, str) and name not in ('|', '')
    else f'col_{i:02d}'
    for i, name in enumerate(colnames_96)
]

# Print key columns for verification
print("Key microlensing parameter columns:")
print(f"  Column 30 (u0):    {colnames_96[30]}")
print(f"  Column 32 (t0):    {colnames_96[32]}")
print(f"  Column 33 (tE):    {colnames_96[33]}")
print(f"  Column 36 (piE):   {colnames_96[36]}")
print(f"  Column 37 (rhos):  {colnames_96[37]}")

In [ ]:
#@title Putting everything in a tidy data frame
master_df = read_table(
    MASTER_SOURCE,
    sep=r'\s+',
    comment='#',
    header=None,
    names=colnames_96,
    engine='python'
)

df_sl = (
    master_df[master_df['sim type'] == 'dcnormffp']
    .copy()
    .reset_index(drop=True)
)

df_sl['lc_number'] = df_sl['lc_number'].astype(int)
df_sl['event_id'] = df_sl['lc_number'].apply(lambda n: f"ulwdc1_{n:03d}")

df_sl[['event_id', 'lc_number', 'N', 'Delta chi2']].head()

<!--We deliberately use `pandas.read_csv(..., delim_whitespace=True)` to parse the catalog so you don’t need to wrestle with `numpy.genfromtxt`. -->
The helper functions above handle both local files and S3 URIs, which means the exact same code path works on Colab, your laptop, or the Nexus.

Each row now includes the `event_id` (e.g., `ulwdc1_123`) and the numeric `lc_number`, so we can fetch the corresponding light curves from either the local `data-challenge-1/lc/` directory or the S3 bucket. We'll construct those references next.


In [ ]:
#@title Figuring out which files we want
def make_local_lightcurve(event_id: str, band: str) -> str:
    return str(LOCAL_LC_DIR / f"{event_id}_{band}.txt")

if LOCAL_LC_DIR.exists():
    df_sl['lc_file_path_W149'] = df_sl['event_id'].apply(lambda eid: make_local_lightcurve(eid, 'W149'))
    df_sl['lc_file_path_Z087'] = df_sl['event_id'].apply(lambda eid: make_local_lightcurve(eid, 'Z087'))
else:
    print("Local light-curve files are not present; use the S3 URIs instead.")

df_sl['lc_s3_uri'] = df_sl['event_id'].apply(lambda eid: S3_LC_TEMPLATE.format(event_id=eid))

preview_cols = ['event_id', 'lc_s3_uri']
if 'lc_file_path_W149' in df_sl.columns:
    preview_cols.insert(1, 'lc_file_path_W149')

df_sl[preview_cols].head()

There are a few pieces of information that do **not** live inside the light curve files (sky position, extinction estimates, etc.). They are provided in `event_info.txt`, which is available both locally (`data-challenge-1/event_info.txt`) and at `s3://rmdc26-data-public/2018-test/event_info.txt`.

Columns: `"Event_name"` `"Event_number"` `"RA_(deg)"` `"Dec_(deg)"` `"Distance"` `"A_W149"` `"sigma_A_W149"` `"A_Z087"` `"sigma_A_Z087"`

`Distance` plus the extinction columns give you approximate clump-star distances and reddening along the line of sight; the sigma values represent the dispersion in each band.


In [ ]:
#@title Event information data frame

header = [
    "Event_name",
    "Event_number",
    "RA_(deg)",
    "Dec_(deg)",
    "Distance",
    "A_W149",
    "sigma_A_W149",
    "A_Z087",
    "sigma_A_Z087"
]

event_info = read_table(
    EVENT_INFO_SOURCE,
    names=header,
    sep=r'\s+',
    engine='python'
)
event_info.head()

In [ ]:
#@title Combining the two data frames

# Convert 'lc_number' to numeric type before merging
merged_df = pd.merge(event_info, df_sl.astype({'lc_number': 'int64'}), left_on='Event_number', right_on='lc_number', how='inner')

# Verify that the key microlensing columns are present
required_cols = ['t0', 'u0', 'tE', 'piE', 'rhos']
missing_cols = [c for c in required_cols if c not in merged_df.columns]

if missing_cols:
    print(f"WARNING: Missing columns: {missing_cols}")
    print("Available columns:", list(merged_df.columns))
else:
    print("All required microlensing columns present.")
    # Display a preview of key microlensing parameters
    print("\nMicrolensing parameters for first 5 events:")
    display_cols = ['event_id', 't0', 'u0', 'tE', 'piE', 'rhos']
    print(merged_df[display_cols].head().to_string(index=False))

In [ ]:
try:  # this will only work on Colab.
  sheet = sheets.InteractiveSheet(df=merged_df)
except:
  # don't limit the number of columns displayed by pandas
  pd.set_option('display.max_columns', None)
  merged_df.head()

Great—data successfully wrangled. Time to pivot from OGLE to the simulated Roman season.

### Adjusting the Model for an L2 Orbit

Space-based light curves require ephemerides so `MulensModel` can evaluate the observer position relative to the lens. For the Roman DC1 data we simply pass `ephemerides_file='data-challenge-1/wfirst_ephemeris_W149.txt'` when instantiating each `MulensData` object.

> Instructions specific to this data set for `MulensModel` are given [here](https://github.com/rpoleski/MulensModel/blob/master/documents/data_challenge.md).

Most of the data for these events is in the W149 band, so we focus on that passband and avoid juggling multiple $(F_s, F_b)$ pairs. If you later need the source color, hold the microlensing parameters fixed and refit only the fluxes (linear regression) in each band—fast and reliable.


In [ ]:
#@title Example L2 data object

# Select the first event from our merged dataframe
event_idx = 0  # Change this to select a different event

# Get the light curve file path
data_file = merged_df['lc_file_path_W149'].iloc[event_idx]
event_id = merged_df['event_id'].iloc[event_idx]

# Extract microlensing parameters from the master file
# These columns were defined during the data loading step
t_0_sim = float(merged_df['t0'].iloc[event_idx])      # time of closest approach (simulation time)
u_0 = float(merged_df['u0'].iloc[event_idx])          # impact parameter
t_E = float(merged_df['tE'].iloc[event_idx])          # Einstein crossing time (days)
pi_E = float(merged_df['piE'].iloc[event_idx])        # parallax amplitude
rho_s = float(merged_df['rhos'].iloc[event_idx])      # source size (rho_star)

# Split the parallax equally between N and E components (simplistic assumption)
# In reality, you'd fit for both components independently
pi_E_E = np.sqrt(pi_E**2 / 2.0)
pi_E_N = pi_E_E

# Sky coordinates
ra = merged_df['RA_(deg)'].iloc[event_idx]
dec = merged_df['Dec_(deg)'].iloc[event_idx]

print(f"Event: {event_id}")
print(f"Sky coordinates: RA={ra:.4f}°, Dec={dec:.4f}°")
print(f"\nTrue parameters from master file:")
print(f"  t0 (sim time): {t_0_sim:.2f}")
print(f"  u0: {u_0:.4f}")
print(f"  tE: {t_E:.2f} days")
print(f"  piE: {pi_E:.6f}")
print(f"  rhos: {rho_s:.6f}")

# Convert RA/Dec to SkyCoord format for MulensModel
coord = SkyCoord(ra=ra * u.deg, dec=dec * u.deg, frame='icrs')
hms_dms_string = coord.to_string('hmsdms')
print(f"\nSkyCoord: {hms_dms_string}")

# Here is the main difference for space-based data - we provide the ephemeris for Roman:
EPHEM_FILE = 'data-challenge-1/wfirst_ephemeris_W149.txt'
data_Roman_W149 = MulensModel.MulensData(file_name=data_file,
                                         phot_fmt='mag',
                                         ephemerides_file=EPHEM_FILE,
                                         plot_properties={'color': '#a859e4',
                                                          'label': 'Roman W149'
                                                          },
                                         bandpass='H'
                                         )

# Convert t_0 from simulation time to HJD
# Reference: https://github.com/microlensing-data-challenge/evaluation_code/blob/master/parse_table1.py
# The simulation zero time corresponds to HJD 2458234.0
t_0 = t_0_sim + 2458234.0

# Build the parameter dictionary for MulensModel
# We multiply by 1.1 to create "guess" parameters slightly offset from truth
params = dict()
parameters_to_fit = ["t_0", "u_0", "t_E", "rho", "pi_E_N", "pi_E_E"]
params['t_0'] = t_0
params['t_0_par'] = t_0  # parallax reference time (often same as t_0)
params['u_0'] = u_0 * 1.1
params['t_E'] = t_E * 1.1
params['rho'] = rho_s * 1.1
params['pi_E_N'] = pi_E_N
params['pi_E_E'] = pi_E_E

print(f"\nInitial guess parameters (true × 1.1):")
print(f"  t_0: {params['t_0']:.2f} (HJD)")
print(f"  u_0: {params['u_0']:.4f}")
print(f"  t_E: {params['t_E']:.2f} days")
print(f"  rho: {params['rho']:.6f}")
print(f"  pi_E_N: {params['pi_E_N']:.6f}")
print(f"  pi_E_E: {params['pi_E_E']:.6f}")

# Create the MulensModel with parallax
# Note: Providing coords and ephemerides_file is essential for parallax calculations
Roman_model = MulensModel.Model({**params},
                                coords=coord,
                                ephemerides_file=EPHEM_FILE
                                )

Roman_event = MulensModel.Event(datasets=data_Roman_W149, model=Roman_model)
print(f"\nMulensModel Event created successfully for {event_id}")

In [ ]:
#@title Sneaky peak at the data

print(' ', data_file, '\n')
! head -5 data-challenge-1/lc/ulwdc1_005_W149.txt

print('\n', EPHEM_FILE)
! head -5 data-challenge-1/wfirst_ephemeris_W149.txt

print('\n Object lightcurve dates')
print(data_Roman_W149.time)

print('\n t_0:', params['t_0'])
print(' t_0_par:', params['t_0_par'])

# cool, everything is in full HJD and looking sensible

In [ ]:
#@title Plot to check it's working

Roman_event = MulensModel.Event(datasets=data_Roman_W149, model=Roman_model)

t_min = params['t_0'] - 3.0 * params['t_E']
t_max = params['t_0'] + 3.0 * params['t_E']
print(t_min, t_max)
print(t_0)

# set finite source method
Roman_model.set_magnification_methods([t_min,
                                       finite_source_methods[0],
                                       t_max],
                                      source=None
                                      )  # rho <= 0.1

plt.close(200)
plt.figure(200)

#Roman_model.plot_magnification(t_range=[t_min, t_max],
#                                    subtract_2450000=True,
#                                    color='red',
#                                    linestyle=':'
#                                    )
#Roman_event.plot_data(subtract_2450000=True)
Roman_event.plot()

plt.xlim(t_min-2450000, t_max-2450000)
plt.show()

Yeah... not great. But good enough for a starting guess, if that's the route you're taking.

### Run the Fits


This is it. You have every thing you need to fit a full season of single lenses. And this is the part where we push the baby bird out of the nest. No more hand-holding. No sample answers. Just do it.

We believe in you.

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 10</h2>
    <p>Perform FSPL on the provided simulated Roman single-lens events.</p>
    <p><i>Note. Don't forget to save your best fit parameters for later inspection and to add parallax to the model.</i></p><br>
</div>

In [ ]:
#@title Your code goes here

In [ ]:
#@title Exercise 10 fit parameter distributions

fit7_params = fit_params[:N].copy()
plot_histograms(fit7_params, 8)

### Evaluate Your Results


The next step is evalutaing how well your bulk fit went. Some simple histograms of fitted parameters compared with "truths" should do the trick.

<style>
.exercise {
    background-color: #E0E0E0;
    border-left: 8px solid #808080;
    padding: 10px 0 10px 20px;  /* top, right, bottom, left */
    margin: 20px 5px;  
    box-sizing: border-box;  
}
.exercise h2 {
    color: #808080;
    font-size: 24px;
}
.exercise p {
    margin: 0 20px;  /* Adjust this value to add space after the paragraph */
}
</style>

<div class="exercise">
    <h2>Exercise 11</h2>
    <p>Make overlayed histograms of the true parameters distributions and your best-fit parameter distributions to evaluate the sucess of your algoryhthms.</p>
    <br>
</div>

In [ ]:
#@title Your code goes here

## Exercises and Next Steps

- Re-run Exercises 1–4 but swap in your own priors or alternative optimizers; record how sensitive the recovered $t_E$ and $u_0$ distributions are.
- Extend Exercise 5 by writing a simple event-finding or anomaly-finding heuristic.
- In Section 4, batch-fit 10–20 Roman events and log runtime/memory usage—this helps you budget future Nexus jobs.
- Package one of your fits with `microlens-submit` to confirm you can validate and export a submission artifact.
- Move on to the next notebook in this series: [`Fitting_Binary_Lenses.ipynb`](../Session%20C:%20Binary%20Lens/Fitting_Binary_Lenses.ipynb).

When you are ready for the real challenge, the Roman Microlensing Data Challenge release can be found on the Nexus s3 bucket (`s3://rmdc26-data-public/`) or on Hugging Face.

## Additional Resources
- [MulensModel documentation](https://rpoleski.github.io/MulensModel/) — API details, geometry references, and Roman-specific notes.
- [OGLE Early Warning System](https://ogle.astrouw.edu.pl/ogle4/ews/ews.html) — daily alert tables with PSPL estimates.
- [microlens-submit docs](https://microlens-submit.readthedocs.io/en/latest/) — workflow for packaging and validating challenge submissions.
- [Roman Research Nexus](https://roman.ipac.caltech.edu/nexus) — information about kernels, data access, and team spaces.
- [RGES-PIT Microlensing resources](https://rges-pit.org/outreach/) — minicourse videos and supplementary tutorials referenced in this session.

## About this Notebook
**Author(s):** Amber Malpas, Katarzyna Kruszyńska, Somayeh Khakpash, Ali Crisp, Meet J. Vyas  
**Maintainers:** RGES-PIT Working Group 9  
**Last Updated:** 20 Nov 2025  
**Contact:** malpas.1@osu.edu

## Citations

If you use `MulensModel`, `emcee`, or this notebook for published research, please cite the
authors. Follow these links for more information about citing:

* [Citing `MulensModel`](https://github.com/rpoleski/MulensModel/blob/master/CITATION.cff)
* [Citing `emcee`](https://github.com/dfm/emcee#attribution)
* [Citing **Roman Microlensing Data Challenge 2026 Notebooks**](https://github.com/rges-pit/data-challenge-notebooks/blob/main/zenodo.txt)


[Top of Page](#top)
<!-- Footer Start -->

<!-- Footer End -->